## Assignment 3

#### Import Required libraries

In [7]:
import os
import matplotlib as mpl
import numpy as np
import pandas as pd
import csv
import chess
import chess.pgn
import io

#### Import csv file

In [8]:
# todo: i forgor what the exact file is called, update this
CHESS_DATA_LOCATION = "data/club_games_data.csv"
# MOVIES_DATA_LOCATION = "data/movies.csv"

# load csv into a big dataframe
chess_data = pd.read_csv(CHESS_DATA_LOCATION)
# print(chess.head())

In [9]:
print(chess_data.dtypes)
# print(chess["rules"].unique())
# ['chess' 'chess960' 'threecheck' 'crazyhouse' 'kingofthehill']
# dunno if we should filter by chess only

# time control
#['1/259200' '1/172800' '1800' '1/86400' '1/432000' '1/604800' '600'
#  '120+1' '900+10' '300' '180+2' '3600+5' '2700+45' '3600' '1/1209600'
#  '180' '600+10' '60' '480+3' '300+5' '420+3' '600+5' '600+2' '1200' '30'
#  '60+1' '120' '1500+3' '900+2' '1500+5' '1500+10' '1/864000' '900' '300+2'
#  '1500' '7200' '300+1' '5400' '3600+60' '2700+30' '3480+45' '10' '2700+10'
#  '15' '2700' '3600+20' '4500' '4200' '900+5' '1800+10' '2700+5' '480+5'
#  '1800+30' '300+3' '600+1' '1800+5' '420+5' '5400+30' '240+10' '420' '303'
#  '60+10']
# so many types of time control -_-

# time class
# ['daily' 'rapid' 'bullet' 'blitz']

white_username    object
black_username    object
white_id          object
black_id          object
white_rating       int64
black_rating       int64
white_result      object
black_result      object
time_class        object
time_control      object
rules             object
rated               bool
fen               object
pgn               object
dtype: object


#### Clean data
Filter out NaN
Lila come back here, can't think of anything else for now

In [10]:
# Drops rows if any value is a NaN
chess_data.dropna(axis = 0, how = 'any')

,white_username,black_username,white_id,black_id,white_rating,black_rating,white_result,black_result,time_class,time_control,rules,rated,fen,pgn
0,-Amos-,miniman2804,https://api.chess.com/pub/player/-amos-,https://api.chess.com/pub/player/miniman2804,1708,1608,win,checkmated,daily,1/259200,chess,True,r2r4/p2p1p1p/b6R/n1p1kp2/2P2P2/3BP3/PP5P/4K2R ...,"[Event ""Enjoyable games 2 - Round 1""]\n[Site ""..."
1,-Amos-,koltcho69,https://api.chess.com/pub/player/-amos-,https://api.chess.com/pub/player/koltcho69,1726,1577,win,resigned,daily,1/172800,chess,True,8/5Q1k/4n1pp/8/7P/2N2b2/PP3P2/5K2 b - - 1 33,"[Event ""Rapid Rats - Board 5""]\n[Site ""Chess.c..."
2,-Amos-,enhmandah,https://api.chess.com/pub/player/-amos-,https://api.chess.com/pub/player/enhmandah,1727,842,win,resigned,daily,1/172800,chess,True,rn1q1b1r/kb2p1pp/2p5/p1Q5/N1BP2n1/4PN2/1P3PPP/...,"[Event ""CHESS BOARD CLASH - Round 1""]\n[Site ""..."
3,enhmandah,-Amos-,https://api.chess.com/pub/player/enhmandah,https://api.chess.com/pub/player/-amos-,819,1727,checkmated,win,daily,1/172800,chess,True,r3kb1r/pp3ppp/3p1n2/2pKp3/P3P3/1P6/4qP1P/QNB5 ...,"[Event ""CHESS BOARD CLASH - Round 1""]\n[Site ""..."
4,-Amos-,Shalllow-Blue,https://api.chess.com/pub/player/-amos-,https://api.chess.com/pub/player/shalllow-blue,1729,1116,win,resigned,daily,1/172800,chess,True,r3b2r/pp6/2pPpR1k/4n3/2P3Q1/3B4/PP4PP/R5K1 b -...,"[Event ""CHESS BOARD CLASH - Round 1""]\n[Site ""..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66874,Alyushkov,abdullahnasib,https://api.chess.com/pub/player/alyushkov,https://api.chess.com/pub/player/abdullahnasib,866,901,resigned,win,blitz,300,chess,True,1k1q1bnr/p1pb2pp/Bp2p3/3pPp2/3P4/N1P2N2/1P3PPP...,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat..."
66875,abdullahnasib,Nizaamulmulk,https://api.chess.com/pub/player/abdullahnasib,https://api.chess.com/pub/player/nizaamulmulk,910,912,win,timeout,blitz,300,chess,True,8/p7/k1p5/1pP4p/1K1P4/1PN1Q3/1r3n1q/8 b - -,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat..."
66876,esadot,abdullahnasib,https://api.chess.com/pub/player/esadot,https://api.chess.com/pub/player/abdullahnasib,909,919,abandoned,win,blitz,300,chess,True,3r1rk1/5p2/1PRp2p1/2nNp2p/4P3/3P2PR/3b2K1/8 w - -,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat..."
66877,abdullahnasib,FrancescoPops,https://api.chess.com/pub/player/abdullahnasib,https://api.chess.com/pub/player/francescopops,928,933,win,resigned,blitz,300,chess,True,r3kb1r/pp1q1p2/2npbN1p/2p1p1p1/2P1P3/3P2PN/PP3...,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat..."


#### Get pgn from last column - can we import things for this?
Idk if it works yet cos am not actually using the file
Lila come back to this

Lila see if can do by getting to '] {new line} 1. ' in one swoop, instead of finding '[Link ' then '1. '

In [11]:
# Save pgn column from dataframe
full_pgn = chess_data['pgn']

def get_moves(entry):
    '''
    Retrive series of moves in a game when given the whole full_pgn entry
    '''
    # # Find last occurrence of ']' in each full_pgn entry
    # link_bracket = entry.rindex('[Link ')

    # # Cut non-move info of full_pgn
    # pgn = full_pgn[link_bracket ::]

    # # Find first occurrence of '1. ', signifying first move, in each full_pgn entry
    # first_move_index = pgn.rindex('1. ')


    # pgn is just a string "separated" by newlines so turn into an array, and pgn is
    # always the final entry in the array. though turning into array may not be the most
    # efficient maybe? not sure tho  - leon
    pgn = entry.splitlines()[-1]
    return pgn

chess_data['move_list'] = full_pgn.apply(get_moves)
print(chess_data['move_list'][134])
# get_moves(full_pgn[0])
# ouput: '1. d4 Nf6 2. c4 e6 3. Nc3 Bb4 4. Qb3 Bxc3+ 5. Qxc3 O-O 6. Bg5 c5 7. dxc5 Nc6 8. Nf3 Qa5 9. Bxf6 gxf6 10. Qxa5 Nxa5 11. e3 Rd8 12. Rd1 Kg7 13. Be2 b6 14. Rd4 bxc5 15. Rg4+ Kh6 16. Bd3 f5 17. Rh4+ Kg6 18. g4 Ba6 19. gxf5+ exf5 20. Ne5+ Kf6 21. Rh6+ Kxe5 22. f4# 1-0'

1. d4 {[%clk 0:09:48]} 1... Nf6 {[%clk 0:09:58.3]} 2. Nc3 {[%clk 0:09:45.4]} 2... d5 {[%clk 0:09:57.2]} 3. e4 {[%clk 0:09:44.5]} 3... dxe4 {[%clk 0:09:55.3]} 4. f3 {[%clk 0:09:43.5]} 4... Nc6 {[%clk 0:09:45.5]} 5. fxe4 {[%clk 0:09:37]} 5... Qxd4 {[%clk 0:09:42.5]} 6. Qxd4 {[%clk 0:09:34]} 6... Nxd4 {[%clk 0:09:41.5]} 7. Bb5+ {[%clk 0:09:21.6]} 7... c6 {[%clk 0:09:37.4]} 8. Ba4 {[%clk 0:09:20.6]} 8... b5 {[%clk 0:09:35.6]} 9. Bb3 {[%clk 0:09:19.2]} 9... b4 {[%clk 0:09:32.9]} 10. Na4 {[%clk 0:09:10.2]} 10... e5 {[%clk 0:09:24.7]} 11. Be3 {[%clk 0:09:05.3]} 11... Nxb3 {[%clk 0:09:12.3]} 12. cxb3 {[%clk 0:08:56.7]} 12... Nxe4 {[%clk 0:09:10.4]} 13. Nf3 {[%clk 0:08:48.1]} 13... Bg4 {[%clk 0:09:00.7]} 14. O-O {[%clk 0:08:45.3]} 14... Be7 {[%clk 0:08:54.4]} 15. Nxe5 {[%clk 0:08:40.4]} 15... Be2 {[%clk 0:08:41.1]} 16. Rxf7 {[%clk 0:08:25.6]} 16... Bf6 {[%clk 0:08:20.3]} 17. Bf4 {[%clk 0:08:01.1]} 17... Bxe5 {[%clk 0:08:17.8]} 18. Rf5 {[%clk 0:07:50.3]} 18... Bd4+ {[%clk 0:08:04.6]} 19. Kh1 {[%

#### Import chess info
https://python-chess.readthedocs.io/en/latest/core.html#chess.Board.san


#### En Passant functions
- has_legal_en_passant() tests if en passant capturing would actually be possible on the next move.
- has_pseudo_legal_en_passant()
- has_legal_en_passant()
- is_en_passant(move: Move) Checks if the given pseudo-legal move is an en passant capture.




Use StringIO to parse games from a string. Lila Use this to apply the en passant functions to for each game???

```python
import io
pgn = io.StringIO("1. e4 e5 2. Nf3 *")
game = chess.pgn.read_game(pgn)
```

In [108]:
# chess testing
fen = "6k1/6p1/p3r2p/3Q4/4NK2/P1r2q2/8/8 w - -"
board = chess.Board(fen)
# print(board)

pgntext = "1. d4 d5 2. e3 c6 3. c4 dxc4 4. Bxc4 b5 5. Bb3 a5 6. Qf3 Bb7 7. Bxf7+ Kd7 8. Qf5+ Kc7 9. Nf3 Nh6 10. Qe5+ Kb6 11. a4 bxa4 12. Nc3 Ng4 13. Nxa4+ Ka6 14. Bc4+ Ka7 15. Qc5+ 1-0"

pgn_en = "1. e4 h5 2. e5 f5 3. exf6 b5"

pgn_en2 = "1. d4 h5 2. d5 e5 3. h3"

# pgn = io.StringIO(chess_data['pgn'][134])
pgn = io.StringIO(pgn_en)
game = chess.pgn.read_game(pgn)
board = game.board()

# print(game)

check = False

def check_pgn(pgn_in, check_actual_ep):
    '''
    
    check_actual_ep is True if you want to know whether ep was a move
    Is False if you are just checking whether it could happen at any point
    '''
    pgn = io.StringIO(pgn_in)
    game = chess.pgn.read_game(pgn)
    board = game.board()
    
    precheck = False
    moved = False
    
    for move in game.mainline_moves():
        if (check_actual_ep):
            moved = board.is_en_passant(move)
        else:
            precheck = board.has_legal_en_passant()
            
        
        if moved or precheck:
            return True
        
        board.push(move)
    return False

print(check_pgn(pgn_en, False))

counter = 0
for i in range(60000):
    if (check_pgn(chess_data['pgn'][i], False) == True):
        counter += 1
print(counter)

True
4750


### Number of games with at least 1 possible en-passant move:
## 4750


### Number of games with an actual en-passant move:
## 1566